# Homework 3

# 1

In [2]:
A_1 = np.array([[400, -201], [-800, 401]])
A_2 = np.array([[401, -201], [-800, 401]])
Y = np.array([[200], [400]])

x = np.linalg.solve(A_1, Y)
y = np.linalg.solve(A_2, Y)
print(x)
print(y)

[[-401.5]
 [-800. ]]
[[160600.00000137]
 [320400.00000274]]


In [7]:
#手动实现
eigenvals_A_1 = np.linalg.eigvals(A_1.T @ A_1)
My_Cond_A = np.sqrt(np.max(eigenvals_A_1)/np.min(eigenvals_A_1))
print(My_Cond_A)

2503.004600350619


In [5]:
#调包
Cond_A = np.linalg.cond(A_1)
print(Cond_A)

2503.0046004808573


# 2

我们都知道，指数分布的均值为 $\frac{1}{\lambda}$，方差为 $\frac{1}{\lambda^2}$，所以我们可以得到：

\begin{cases}
\frac{1}{\lambda} = \frac{1}{n} \sum_{i=1}^n x_i \\
\frac{1}{\lambda^2} = \frac{1}{n} \sum_{i=1}^n x_i^2 \\
\end{cases}

如果只用一阶矩，则矩估计为：
$$\hat{\lambda} = \frac{n}{\sum_{i=1}^n x_i}$$

如果用两阶矩，则矩估计为：
$$\hat{\lambda} = \sqrt{\frac{n}{\sum_{i=1}^n x_i^2}}$$

# 3

显然：

$$
\bar {\hat y} = \frac{1}{n} \sum_{i=1}^n \hat y_i = \frac{1}{n} \sum_{i=1}^n x_i^\prime \beta + \frac{1}{n} \sum_{i=1}^n \epsilon_i = \frac{1}{n} \sum_{i=1}^n x_i^\prime \beta = \bar y
$$

# 4

\begin{align*}
\ln L(\beta, \sigma^2) &= -\frac{n}{2} \ln (2\pi) - \frac{n}{2} \ln \sigma^2 - \frac{1}{2\sigma^2} \sum_{i=1}^n (y_i - x_i^\prime \beta)^2 \\
\end{align*}

对其求导得：
\begin{align*}
\frac{\partial \ln L(\beta, \sigma^2)}{\partial \beta} &= \frac{1}{\sigma^2} \sum_{i=1}^n x_i (y_i - x_i^\prime \beta) = 0 \\
\frac{\partial \ln L(\beta, \sigma^2)}{\partial \sigma^2} &= -\frac{n}{2\sigma^2} + \frac{1}{2\sigma^4} \sum_{i=1}^n (y_i - x_i^\prime \beta)^2 = 0 \\
\end{align*}

解得：
\begin{align*}
\hat \beta &= (X^\prime X)^{-1} X^\prime y \\
\hat \sigma^2 &= \frac{1}{n} \sum_{i=1}^n (y_i - x_i^\prime \hat \beta)^2 \\
\end{align*}

# 5

工具变量用于解决解释变量的内生性问题，其需要满足两个条件：

1、工具变量与解释变量相关（相关性）

2、工具变量与误差项不相关（外生性）


我感觉题目没写清楚，我姑且认为这里的交换指的是特征维度上的交换。

不妨记$\tilde X = XP$，其中$P$为一个置换矩阵。

则：
\begin{align*}
\beta_{IV} &= (\tilde X^\prime X)^{-1} \tilde X^\prime Y \\
&= ((XP)^\prime X)^{-1} (XP)^\prime Y \\
&= (P^\prime X^\prime X)^{-1} P^\prime X^\prime Y \\
&= (X^\prime X)^{-1} {P^\prime}^{-1} P^\prime X^\prime Y \\
&= (X^\prime X)^{-1} X^\prime Y \\
&= \beta_{os} \\
\end{align*}


# 6

In [28]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

n = 50
t = np.arange(1, n+1) + np.random.uniform(-0.01, 0.01, n)
X = np.array([np.ones_like(t), t, t**2]).copy().T

In [29]:
def VIF(X):
    vif = []
    for i in range(X.shape[1]):
        y = X[:, i].copy()
        x = np.delete(X, i, axis=1).copy()
        r2 = LinearRegression(fit_intercept=False).fit(x, y).score(x, y)
        vif.append(1/(1-r2))
    return vif

VIF(X)

[1.0, 16.628245953376705, 16.628245953376673]

故$t$和$t^2$有较强的共线性，我们不妨丢掉$t^2$。

In [30]:
X_now = np.delete(X, 2, axis=1).copy()
VIF(X_now)

[1.0, 0.9999999999999998]

# 7

在Ridge回归中：

\begin{align*}
\beta &= argmin_\beta \sum_{i=1}^n (y_i - x_i^\prime \beta)^2 + \frac{1}{2}\lambda \sum_{j=1}^p \beta_j^2 \\
\end{align*}

求导易得：
$$
\beta = (X^\prime X + \lambda I)^{-1} X^\prime y
$$

# 8

不妨只考虑logit模型，且只考虑二分类问题：

我们记$\sigma(x) = \frac{1}{1 + e^{-x}}$，其导数为$\sigma^\prime(x) = \sigma(x) (1 - \sigma(x))$。

此时我们需要回归的是：

$$
\mathbb P[Y=1|X] = \sigma(X^\prime \beta)
$$

如果我们认为$Y$服从伯努利分布，则其似然函数为：
\begin{align*}
L(\beta) &= \prod_{i=1}^n \mathbb P[Y=y_i|X=x_i] \\
&= \prod_{i=1}^n \mathbb P[Y=1|X=x_i]^{y_i} \mathbb P[Y=0|X=x_i]^{1-y_i} \\
&= \prod_{i=1}^n \sigma(x_i^\prime \beta)^{y_i} (1 - \sigma(x_i^\prime \beta))^{1-y_i} \\
\end{align*}

其对数似然函数为：
\begin{align*}
\ln L(\beta) &= \sum_{i=1}^n y_i \ln \sigma(x_i^\prime \beta) + (1-y_i) \ln (1 - \sigma(x_i^\prime \beta)) \\
\end{align*}


其导数为:
\begin{align*}
\frac{\partial \ln L(\beta)}{\partial \beta} &= \sum_{i=1}^n y_i \frac{\partial \ln \sigma(x_i^\prime \beta)}{\partial \beta} + (1-y_i) \frac{\partial \ln (1 - \sigma(x_i^\prime \beta))}{\partial \beta} \\
&= \sum_{i=1}^n y_i \frac{\sigma^\prime(x_i^\prime \beta)}{\sigma(x_i^\prime \beta)} x_i + (1-y_i) \frac{-\sigma^\prime(x_i^\prime \beta)}{1 - \sigma(x_i^\prime \beta)} x_i \\
&= \sum_{i=1}^n y_i (1 - \sigma(x_i^\prime \beta)) x_i - (1-y_i) \sigma(x_i^\prime \beta) x_i \\
&= \sum_{i=1}^n (y_i - \sigma(x_i^\prime \beta)) x_i \\
\end{align*}

进一步其二阶导数为：
\begin{align*}
\frac{\partial^2 \ln L(\beta)}{\partial \beta \partial \beta^\prime} &= \sum_{i=1}^n -\sigma^\prime(x_i^\prime \beta) x_i x_i^\prime \\
\end{align*}

其为负半定矩阵，故其为凹函数，很容易求解其最大值。（具体而言，使用梯度下降或牛顿法即可）